In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = False

In [3]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

  0%|          | 0/4 [00:00<?, ?it/s]

0                                                                                      Gabrielle/
1                                                                                Gabrielle/Orders
2                                                                           Gabrielle/Orders/AOIs
3                                         Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                              Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                                   ...                                           
54841    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
54842    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
54843    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
54844    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
54845               

In [4]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29906,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29907,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29910,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29911,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29918,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
108393,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
108399,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
109500,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
109501,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_13FEB2021.shp


In [5]:
image_extensions = (".jpg", ".jp2", ".tif", ".TIF", ".png")

In [6]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith(image_extensions)]
images

1083      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
1084      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
1086        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
1089        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
1103        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
109733            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
109739            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
109742            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
109743            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
109746            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Length: 2649, dtype: object

In [7]:
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith(image_extensions)]

In [8]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    else:
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=images, processor=fuzz_preprocess)
        return match, score

df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

/tmp/ipykernel_2325973/4132008065.py:24: TqdmWarning: Iterable length 2443 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))


  0%|          | 0/2443 [00:00<?, ?it/s]

Perfect matches: 2044
Imperfect matches: 399


In [9]:
df = df.sort_values("match_score")
df

,filename,matched_image,match_score
30254,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31110,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30413,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
30201,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
30417,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_02JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/TeKaroBay/TeKaroBay_03JAN2022.tif,62.857143
...,...,...,...
4690,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_18SEP1981.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_18SEP1981_mosaic.jp2,100.000000
4693,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_10OCT1973.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_10OCT1973_mosaic.jp2,100.000000
4701,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_25FEB1938.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_25FEB1938_mosaic.jp2,100.000000
4428,Retrolens/HawkesBay/Whirinaki/Shorelines/Whirinaki_26SEP1972.shp,Retrolens/HawkesBay/Whirinaki/Stack/Whirinaki_26SEP1972_mosaic.jp2,100.000000


In [10]:
def get_shapefile_meta(tup):
    i, row = tup
    filename = row.filename
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return {"Source": source, "n_lines": 0}
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    CPS = ""
    if "CPS" not in shapefile.columns:
        CPS = "No CPS column"
    else:
        CPS = shapefile.CPS.values
        if len(CPS) == 0:
            CPS = "No CPS values"
        elif 9 in CPS:
            CPS = "Has CPS=9"
        elif 0 in CPS:
            CPS = "Has CPS=0"
        elif None in CPS:
            CPS = "Has None CPS"
        elif pd.isna(CPS).any():
            CPS = "Has NA CPS"
        else:
            CPS = "OK"
    return {
        "Source": source,
        "n_lines": n_lines,
        "CPS": CPS,
        "Photoscale": "Photoscale" in shapefile.columns,
        "Pixel_Er": "Pixel_Er" in shapefile.columns
    }

shp_meta = pd.DataFrame(process_map(get_shapefile_meta, df.iterrows(), total=len(df)), index=df.index)
df = pd.concat([df, shp_meta], axis=1)
df

  0%|          | 0/2443 [00:00<?, ?it/s]

Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp has ambiguous sources: ['LDS' None]
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Tasman/MoutereRiver/Shorelines/MoutereRiver_13SEP1985.shp has ambiguous sources: ['RL' None]
Retrolens/WestCoast/Westport/Shorelines/Westport_15JAN1988.shp has ambiguous sources: ['Rl' 'RL']
Retrolens/Northland/BayleysBeach/Shorelines/BayleysBeach_07APR2023.shp has ambiguous sources: ['MAX' None]
MaxarImagery/HighFreq/Northland/MarsdenPoint/Shorelines/MarsdenPoint_01JAN2020.shp has ambiguous sources: ['MAX' None]
Retrolens/Southland/HaldaneBay/Shorelines/HaldaneBay_07OCT1985.shp has ambiguous sources: ['RL' None]
Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_24OCT1953.shp has ambiguous sources: ['RL' None]
MaxarImagery/HighFreq/Wellington/KapitiMid/Shorelines/KapitiMid_31O

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30254,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,OK,True,True
31110,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,OK,True,True
30413,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,OK,True,True
30201,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,OK,True,True
30417,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_02JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/TeKaroBay/TeKaroBay_03JAN2022.tif,62.857143,LDS,29,OK,True,True
...,...,...,...,...,...,...,...,...
4690,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_18SEP1981.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_18SEP1981_mosaic.jp2,100.000000,RL,4,OK,True,True
4693,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_10OCT1973.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_10OCT1973_mosaic.jp2,100.000000,RL,4,OK,True,True
4701,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_25FEB1938.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_25FEB1938_mosaic.jp2,100.000000,RL,8,OK,True,True
4428,Retrolens/HawkesBay/Whirinaki/Shorelines/Whirinaki_26SEP1972.shp,Retrolens/HawkesBay/Whirinaki/Stack/Whirinaki_26SEP1972_mosaic.jp2,100.000000,RL,5,OK,True,True


In [11]:
df.CPS.value_counts()

CPS
OK               2297
No CPS column      85
Has CPS=9          19
No CPS values      16
Has CPS=0          13
Has None CPS        7
Has NA CPS          6
Name: count, dtype: int64

In [26]:
df[(df.CPS!="OK") & (df.n_lines > 0)][["filename", "Source", "n_lines", "CPS"]].sort_values(by=["CPS", "filename"]).to_csv(prefix+"Nick/CPS_issues.csv", index=False)

In [13]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30254,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,OK,True,True
31110,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,OK,True,True
30413,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,OK,True,True
30201,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,OK,True,True
30417,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_02JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/TeKaroBay/TeKaroBay_03JAN2022.tif,62.857143,LDS,29,OK,True,True
...,...,...,...,...,...,...,...,...
85845,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_17MAR1961.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_17MAR1961.tif,100.000000,RL,27,OK,True,True
85839,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_22AUG1971.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_22AUG1971.tif,100.000000,RL,12,OK,True,True
85838,Retrolens/Northland/BayofIslands_East/Shorelines/Bayofislands_East_29MAR1951.shp,Retrolens/Northland/BayofIslands_East/Stack/BayofIslands_East_29MAR1951.tif,100.000000,RL,50,OK,True,True
86497,Retrolens/Northland/NorthHeadKaipara/Shorelines/NorthHeadKaipara_11MAY1938.shp,Retrolens/Northland/NorthHeadKaipara/Stack/NorthHeadKaipara_11MAY1938.tif,100.000000,RL,6,OK,True,True


In [14]:
failures = (df.match_score < 100) & (df.Source != "LDS") & (df.n_lines > 0)
print(sum(failures))
try:
  df[failures].to_csv(prefix+"Nick/failures.csv", index=False)
except:
  print("Can't write Nick/failures.csv")
df[failures]

1


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
96563,Retrolens/Gisborne/Tolaga_KaiauaBay/Shorelines/Tolaga_KaiauaBay_14MAR1943.shp,Retrolens/Gisborne/Tolaga_KaiauaBay/Stack/Tolaga_KaiauaBay_04MAR1979_mosaic.jp2,89.361702,RL,2,OK,True,True


In [15]:
df.to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

### For those perfect matches, get the metadata from the corresponding matched image

In [16]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return {}
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return {}
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/2044 [00:00<?, ?it/s]

/home/nyou045/.local/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,interleave,GCPs,res,compress,photometric
0,Retrolens/Otago/NorthOamaru/Shorelines/NorthOamaru_09MAY1955.shp,Retrolens/Otago/NorthOamaru/Stack/NorthOamaru_09MAY1955_mosaic.jp2,100.0,RL,3,True,True,True,JP2OpenJPEG,uint16,256.0,17865,26846,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.49811576733726926, 0.0, 1445198.3788442442, 0.0, -0.49811576733727553, 5022177.283355617, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.49811576733726926, 0.49811576733727553)",NaN,NaN
1,Retrolens/Otago/CoalPoint_SmithsBeach/Shorelines/CoalPoint_SmithsBeach_20FEB1975.shp,Retrolens/Otago/CoalPoint_SmithsBeach/Stack/CoalPoint_SmithsBeach_20FEB1975_mosaic.jp2,100.0,RL,11,True,True,True,JP2OpenJPEG,uint16,256.0,9522,8052,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.7111884325875153, 0.0, 1360345.9266482175, 0.0, -0.7111884325875041, 4871576.50750212, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.7111884325875153, 0.7111884325875041)",NaN,NaN
2,Retrolens/Otago/Jacks_Surat_CannibalBay/Shorelines/Jacks_Surat_CannibalBay_21FEB1967.shp,Retrolens/Otago/Jacks_Surat_CannibalBay/Stack/Jacks_Surat_CannibalBay_21FEB1967_mosaic.jp2,100.0,RL,3,True,True,True,JP2OpenJPEG,uint16,256.0,2431,2449,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.8333099079631532, 0.0, 1347144.582911074, 0.0, -1.833309907963208, 4849381.133461215, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(1.8333099079631532, 1.833309907963208)",NaN,NaN
3,Retrolens/Otago/CoalPoint_SmithsBeach/Shorelines/CoalPoint_SmithsBeach_26FEB1946.shp,Retrolens/Otago/CoalPoint_SmithsBeach/Stack/CoalPoint_SmithsBeach_26FEB1946_mosaic.jp2,100.0,RL,16,True,True,True,JP2OpenJPEG,uint16,256.0,12817,11235,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5119569218150053, 0.0, 1360413.1974775917, 0.0, -0.5119569218150101, 4871533.459767245, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.5119569218150053, 0.5119569218150101)",NaN,NaN
4,Retrolens/Otago/Jacks_Surat_CannibalBay/Shorelines/Jacks_Surat_CannibalBay_26FEB1975.shp,Retrolens/Otago/Jacks_Surat_CannibalBay/Stack/Jacks_Surat_CannibalBay_26FEB1975_mosaic.jp2,100.0,RL,3,True,True,True,JP2OpenJPEG,uint16,256.0,6261,6422,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.7029303538772481, 0.0, 1347147.9710873629, 0.0, -0.7029303538773124, 4849414.606463266, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.7029303538772481, 0.7029303538773124)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_18SEP1981.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_18SEP1981_mosaic.jp2,100.0,RL,4,True,True,True,JP2OpenJPEG,uint16,256.0,6555,6642,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.681536670240256, 0.0, 2017524.738734676, 0.0, -0.6815366702402743, 5666118.313826275, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.681536670240256, 0.6815366702402743)",NaN,NaN
2040,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_10OCT1973.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_10OCT1973_mosaic.jp2,100.0,RL,4,True,True,True,JP2OpenJPEG,uint16,256.0,6469,6365,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.7335962908908487, 0.0, 2017637.5365148883, 0.0, -0.7335962908908736, 5666213.660959688, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.7335962908908487, 0.7335962908908736)",NaN,NaN
2041,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_25FEB1938.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_25FEB1938_mosaic.jp2,100.0,RL,8,True,True,True,JP2OpenJPEG,uint16,256.0,7606,8127,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5267034130385344, 0.0, 2017912.0603475603, 0.0, -0.5267

## Read LDS index tiles

In [17]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

4519     Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10081            Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11958         Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13938         Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14057              Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                               ...                                                       
35733             SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
35738        SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp
43343                   

In [18]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/79 [00:00<?, ?it/s]

147407

In [19]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   22679
2022                   11348
2023                    3607
24/02/2017              2685
                       ...  
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
Name: count, Length: 384, dtype: Int64

## LDS index tile matching

In [20]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30254,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,OK,True,True
31110,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,OK,True,True
30413,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,OK,True,True
30201,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,OK,True,True
30417,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_02JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/TeKaroBay/TeKaroBay_03JAN2022.tif,62.857143,LDS,29,OK,True,True
...,...,...,...,...,...,...,...,...
59855,Retrolens/Manawatu/CastleCliff/Shorelines/CastleCliff_10JUN1942_clifftop.shp,Retrolens/Manawatu/CastleCliff/Stack/CastleCliff_10JUN1942_mosaic.jp2,93.023256,LDS,1,OK,True,True
76921,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS,8,OK,True,True
39123,Retrolens/Southland/Tiwai_Point/Shorelines/TiwaiPoint_07FEB2016.shp,MaxarImagery/HighFreq/Southland/TiwaiPoint/Stack/TiwaiPoint_02FEB2016.tif,94.444444,LDS,1,OK,True,True
47680,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_beach.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Stack/Oakura_03OCT2016.tif,96.551724,LDS,3,OK,True,True


In [21]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

#get_resolution("Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_19MAR2017_Taranaki399_cliff.shp")
LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
LDS

  0%|          | 0/398 [00:00<?, ?it/s]

Perfect matches: 289
Imperfect matches: 109


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
43,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
171,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
129,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
330,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
356,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
...,...,...,...,...,...,...,...,...
138,Retrolens/Gisborne/WaiapuRiver/Shorelines/WaiapuRiver_16FEB2018.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2018-02-16,16/02/2018,16/02/2018,FLOWN,100.000000,0.3
137,Retrolens/WestCoast/MikonuiRiver/Shorelines/MikonuiRiver_27FEB2017_WST16R.shp,[SpatialData/Mosaics/Footprint shapefiles/West Coast footprints/west-coast-03m-rural-aerial-photos-index-tiles-2016-2017.shp],2017-02-27,27/02/2017,"8,16,17,23,27/02/2017; 8,9/03/2017",FLOWN,100.000000,0.3
136,Retrolens/Northland/BayleysBeach/Shorelines/BayleysBeach_06NOV2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Northland Footrpints/northland-04m-rural-aerial-photos-index-tiles-2014-2016.shp],2015-11-06,06/11/2015,"06/11/2015, 07/11/2015",DATE_DMY,100.000000,0.4
221,Retrolens/Taranaki/UrenuiRiver_North_AOI/Shorelines/UrenuiRiver_North_7JAN2022_beach.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-025m-rural-aerial-photos-index-tiles-2021-2022.shp],2022-01-07,07/01/2022,07/01/2022,FLOWN,100.000000,0.25


In [22]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
43,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
171,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
129,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
330,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
356,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
0,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,[Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp],05/04/2023,05/04/2023,2023-04-05,FLOWN,44.444444,0.10
256,Retrolens/HawkesBay/OceanBeach/Shorelines/OceanBeach_06MAR2019.shp,[SpatialData/Mosaics/Footprint shapefiles/Hawkes Bay Footprints/hawkes-bay-03m-rural-aerial-photos-index-tiles-2019-2020.shp],2019-03-06,6/03/2019,05/03/2019,FLOWN,44.444444,0.3
299,Retrolens/Otago/Warrington/Shorelines/Warrington_30MAR2006.shp,[SpatialData/Mosaics/Footprint shapefiles/Otago footprints/lds-dunedin-01m-urban-aerial-photos-index-tiles-2018-2019-SHP/dunedin-01m-urban-aerial-photos-index-tiles-2018-2019.shp],1899-12-30,1899-12-30,"07/04/2019,08/02/2019,06/11/2018,14/01/18",FLOWN,45.000000,0.1
393,Retrolens/Manawatu/CastleCliff/Shorelines/CastleCliff_10JUN1942_clifftop.shp,"[SpatialData/Mosaics/Footprint shapefiles/Manawatu/manawatu-whanganui-03m-rural-aerial-photos-index-tiles-2021-.shp, SpatialData/Mosaics/Manawatu 2021/manawatu-whanganui-03m-rural-aerial-photos-in...",1942-06-10,10/06/1942,"17/03/2021, 21/02/2021",FLOWN,45.000000,0.3
315,Retrolens/Taranaki/KaupokonuiBeach/Shorelines/KaupokonuiBeach_16MAR2017.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-03-16,16/03/2017,11/2/2017,FLOWN,47.058824,0.3


In [23]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)